In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import transformers
from transformers import Trainer,TrainingArguments,AutoImageProcessor,AutoModelForImageClassification,EarlyStoppingCallback
import datasets
from datasets import load_dataset
from sklearn.metrics import accuracy_score,precision_score
from collections import Counter

In [ ]:
gc.collect()

In [ ]:
torch.cuda.is_available()

In [ ]:
model_id = 'google/vit-base-patch16-224'

In [ ]:
dataset = load_dataset('aum27/mars-terrain',split='train')

In [ ]:
dataset, dataset[0]

In [ ]:
feature_extractor = AutoImageProcessor.from_pretrained(model_id,trust_remote_code=True)

In [ ]:
def preprocess(batch):
    img = [x.convert('RGB') for x in batch['image']]
    inps = feature_extractor(img,return_tensors='pt')
    batch['pixel_values'] = inps['pixel_values']
    # batch['label'] = batch['label']
    return batch

In [ ]:
dataset = dataset.map(preprocess,batched=True).train_test_split(test_size=0.2, seed=42)

In [ ]:
dataset.set_format(type='torch',columns=['pixel_values','label'])

In [ ]:
train_labels = [dataset['train'][i]['label'].item() for i in range(len(dataset['train']))]
train_label_counts = Counter(train_labels)

val_labels = [dataset['test'][i]['label'].item() for i in range(len(dataset['test']))]
val_label_counts = Counter(val_labels)

print("Training dataset label counts:")
print(train_label_counts)

print("\nValidation dataset label counts:")
print(val_label_counts)

In [ ]:
model = AutoModelForImageClassification.from_pretrained(model_id,num_labels=8,ignore_mismatched_sizes=True,trust_remote_code=True)

In [ ]:
device = torch.device('cuda')
device

In [ ]:
model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir='./output_results_vit',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_checkpointing=True,
    do_eval=True,
    evaluation_strategy='epoch',
    num_train_epochs=5,
    save_strategy='epoch',
    learning_rate=6e-5,
    lr_scheduler_type='linear',
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

In [ ]:
def collate(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }


In [ ]:
def compute_metrics(preds):
    labels = preds.label_ids
    preds = np.argmax(preds.predictions,axis=1)
    acc = accuracy_score(labels,preds)

    prec = precision_score(labels,preds,average='weighted')

    return {'accuracy':acc,'precision':prec}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=collate,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]



)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()